In [109]:
import numpy as np
import pandas as pd
import gc
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight, BM25Recommender
from implicit.als import AlternatingLeastSquares
from implicit.cpu.bpr import BayesianPersonalizedRanking
from implicit.recommender_base import RecommenderBase
from implicit import evaluation
from utils import pandas_df_to_csr
import json
import statistics

In [177]:
BPR_WEIGHT = 1
ALS_WEIGHT = 1
BM25_WEIGHT = 0.5
K = 10

### Load dataset

In [2]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)
# amazon_beauty_csr_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8).tocsr()

In [48]:
amazon_beauty_coo_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

In [70]:
# Test-Train Split
train_csr, test_csr = evaluation.train_test_split(amazon_beauty_coo_bm25, train_percentage=0.8, random_state=55)
print(f"Train size: {train_csr.size} \n Test size: {test_csr.size}")

Train size: 1618938 
 Test size: 404132


In [55]:
train_csr[userid]

<1x249274 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

### Load models

In [125]:
ALS_model = AlternatingLeastSquares(factors=128, regularization=0.1, alpha=3.0)
ALS_model.fit(train_csr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [21:47<00:00, 87.17s/it]


In [51]:
# ALS_model = AlternatingLeastSquares.load("4_CF_ALS_implicit")
BPR_model = BayesianPersonalizedRanking.load("5_CF_BPR_implicit")
BM25_model = BM25Recommender.load("6_BM25")

### Inference - Single User

In [133]:
# Get recommendations for the a single user
userid = 1
results = {}

In [116]:
ids, scores = BPR_model.recommend(userid, train_csr[userid], N=K, filter_already_liked_items=False)
user_dict = {}
user_dict[userid] = {"ids": list(ids), "scores": list(scores)}
results["BPR"] = user_dict
results["BPR"]["weight"] = 1
results

{'BM25': {'ids': [], 'scores': [], 'weight': 1},
 'BPR': {42: {'ids': [69377,
    47485,
    112057,
    17612,
    13646,
    125764,
    99864,
    91742,
    210894,
    51510],
   'scores': [1.0161221,
    0.93109715,
    0.92633134,
    0.8856752,
    0.8801099,
    0.86598194,
    0.8624427,
    0.84489995,
    0.8358341,
    0.8277712]},
  'weight': 1}}

In [134]:
ids, scores = ALS_model.recommend(userid, test_csr[userid], N=K, filter_already_liked_items=False)
results["ALS"] = {"ids": list(ids), "scores": list(scores)} 
results["ALS"]["weight"] = 1
results

{'ALS': {'ids': [9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
  'scores': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  'weight': 1}}

In [115]:
ids, scores = BM25_model.recommend(userid, test_csr[userid], N=K, filter_already_liked_items=False)
results["BM25"] = {"ids": list(ids), "scores": list(scores)} 
results["BM25"]["weight"] = 1
results

{'BM25': {'ids': [], 'scores': [], 'weight': 1}}

In [23]:
def normalize(arr, t_min=0, t_max=1):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)   
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [33]:
def sort_by_score(x):
    return dict(sorted(x.items(), reverse=True, key=lambda item: item[1]))

In [36]:
ensemble_results = {}
for _model, _results in results.items():
    print("Model: ",_model)
    normalized_scores = normalize(arr=_results['scores'], t_max=_results['weight'])
    print(normalized_scores)
    for id, score in zip(_results['ids'], normalized_scores):
        # Case where product is already recommended by one or more other models
        if id in ensemble_results:
            ensemble_results[id] += score # Add the score to the previous value
            ensemble_results[id] /= 2 # Average the score (This is a rough average)
        # Case where product is already recommended first time by current model
        else:
            ensemble_results[id] = score
    
ensemble_results = sort_by_score(ensemble_results)
ensemble_results

Model:  BPR
[1.0, 0.39893320051043213, 0.3034629072782572, 0.18899484393359542, 0.15168046489629314, 0.07546122116979978, 0.07241763199805365, 0.041353837364369185, 0.032233358378842426, 0.0]
Model:  BM25
[1.0, 0.6369881469882088, 0.0655036698212032, 0.06547735511979227, 0.012015765006171138, 0.0036333745798613577, 0.001721750771982628, 0.0013743951596270217, 0.0002476635338376557, 0.0]


{69377: 1.0,
 81854: 1.0,
 154092: 0.6369881469882088,
 13646: 0.39893320051043213,
 47485: 0.3034629072782572,
 17612: 0.18899484393359542,
 125764: 0.15168046489629314,
 239507: 0.07546122116979978,
 99864: 0.07241763199805365,
 0: 0.0655036698212032,
 89013: 0.06547735511979227,
 112057: 0.041353837364369185,
 166611: 0.032233358378842426,
 154097: 0.012015765006171138,
 154084: 0.0036333745798613577,
 154078: 0.001721750771982628,
 137641: 0.0013743951596270217,
 112431: 0.0002476635338376557,
 91742: 0.0,
 196064: 0.0}

### Inference - Test set

In [138]:
results = {}
results["BPR"] = {"weight": BPR_WEIGHT}
results["ALS"] = {"weight": ALS_WEIGHT}
results["BM25"] = {"weight": BM25_WEIGHT}
results["ENSEMBLE"] = {"weight": 1}
results

{'BPR': {'weight': 0.2},
 'ALS': {'weight': 0.8},
 'BM25': {'weight': 1.0},
 'ENSEMBLE': {'weight': 1}}

In [178]:
# Update weights
results["BPR"]["weight"] = BPR_WEIGHT
results["ALS"]["weight"] = ALS_WEIGHT
results["BM25"]["weight"] = BM25_WEIGHT

In [79]:
test_coo = test_csr.tocoo()

In [ ]:
actual_dict = {}
eval_dict = {"precision": [], "recall": [], "f1_score": []}
for user_id, product_id, rating in zip(test_coo.row, test_coo.col, test_coo.data):
    # print(f"Processing: user_id: {user_id}, product_id: {product_id}, rating: {rating}")
    print(user_id, end = ', ')
    # Retrieve actual products and ratings
    if user_id in actual_dict:
        actual_dict[user_id].append(product_id)
    else:
        actual_dict[user_id] = [product_id]

    # Get recommendation from each model
    ids, scores = BPR_model.recommend(user_id, test_csr[user_id], N=K, filter_already_liked_items=False)
    results["BPR"][user_id] = {"product_ids": list(ids), "scores": list(scores)}
    
    ids, scores = ALS_model.recommend(user_id, test_csr[user_id], N=K, filter_already_liked_items=False)
    results["ALS"][user_id] = {"product_ids": list(ids), "scores": list(scores)}

    ids, scores = BM25_model.recommend(user_id, test_csr[user_id], N=K, filter_already_liked_items=False)
    results["BM25"][user_id] = {"product_ids": list(ids), "scores": list(scores)}

    # Ensemble results
    ensemble_results = {}
    for _model, _results in results.items():
        if _model == "ENSEMBLE":
            continue
        scores = _results[user_id]['scores']
        # Check if all product scores are equal or empty
        if len(set(scores)) <= 1:
            print(f"Skipping Model: {_model}, User Id: {user_id}")
            continue
        # Score is normalized to range 0-1 and then weighted by the specified model weight 
        normalized_scores = normalize(arr=scores, t_max=_results['weight'])
        # print("Normalized scores: ", normalized_scores)
        for id, score in zip(_results[user_id]['product_ids'], normalized_scores):
            # Case where product is already recommended by one or more other models
            if id in ensemble_results:
                ensemble_results[id] += score # Add the score to the previous value
                ensemble_results[id] /= 2 # Average the score (This is a rough average)
            # Case where product is recommended first time by current model
            else:
                ensemble_results[id] = score
        
    ensemble_results = sort_by_score(ensemble_results)
    results["ENSEMBLE"][user_id] = {"product_ids": list(ensemble_results.keys()), "scores": list(ensemble_results.values())}

    # Evaluate ensemble results
    actual_products = actual_dict[user_id]
    ensemble_products = results["ENSEMBLE"][user_id]["product_ids"]
    eval_dict["precision"].append(precision_at_k(actual_products, ensemble_products, K))
    eval_dict["recall"].append(recall_at_k(actual_products, ensemble_products, K))
    eval_dict["f1_score"].append(f1_acore_at_k(actual_products, ensemble_products, K))
    
    # break
# print("Actual: ", actual_dict)
# print("Predicted: ", results)


In [180]:
print("Ensemble recommendation results -------")
print("Precision at K: ", statistics.fmean(eval_dict["precision"]))
print("Recall at K: ", statistics.fmean(eval_dict["recall"]))
print("F1 score at K: ", statistics.fmean(eval_dict["f1_score"]))

Ensemble recommendation results -------
Precision at K:  0.11708476438391417
Recall at K:  0.8168642918650342
F1 score at K:  0.1932667989134614


Results with equal weights:
Ensemble recommendation results -------
Precision at K:  0.1151417438713989
Recall at K:  0.879726517120218
F1 score at K:  0.19492122310857204

Ensemble recommendation results -------
BPR_WEIGHT = 0.2
ALS_WEIGHT = 0.8
BM25_WEIGHT = 1.0
Precision at K:  0.11551102115150497
Recall at K:  0.878208357795375
F1 score at K:  0.19484041665986454

Ensemble recommendation results -------
BPR_WEIGHT = 0.5
ALS_WEIGHT = 0.8
BM25_WEIGHT = 1.0
Precision at K:  0.13576925361020659
Recall at K:  0.8605703087110153
F1 score at K:  0.21659103398676482

Ensemble recommendation results -------
BPR_WEIGHT = 1.0
ALS_WEIGHT = 0.8
BM25_WEIGHT = 1.0
Precision at K:  0.13087308107252085
Recall at K:  0.8501083804301565
F1 score at K:  0.21067435575404123

Ensemble recommendation results -------
BPR_WEIGHT = 0.5
ALS_WEIGHT = 0.5
BM25_WEIGHT = 1.0
Precision at K:  0.13801208516029415
Recall at K:  0.8653632971405383
F1 score at K:  0.21932993484321203

Ensemble recommendation results -------
BPR_WEIGHT = 0.5
ALS_WEIGHT = 0.2
BM25_WEIGHT = 1.0
Precision at K:  0.14197440440252196
Recall at K:  0.873360684132907
F1 score at K:  0.22403032030382244

Ensemble recommendation results -------
BPR_WEIGHT = 0.5
ALS_WEIGHT = 0
BM25_WEIGHT = 1.0
Precision at K:  0.14482198885512657
Recall at K:  0.881021547415201
F1 score at K:  0.2278665552786446

Ensemble recommendation results -------
BPR_WEIGHT = 0
ALS_WEIGHT = 0
BM25_WEIGHT = 1.0
Precision at K:  0.15390416992467806
Recall at K:  0.8944998168915107
F1 score at K:  0.23739280949092406

Ensemble recommendation results -------
BPR_WEIGHT = 1
ALS_WEIGHT = 1
BM25_WEIGHT = 0.5
Precision at K:  0.11708476438391417
Recall at K:  0.8168642918650342
F1 score at K:  0.1932667989134614

In [121]:
len(actual_dict)

224212

In [101]:
# Pretty print
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))


In [ ]:
pretty(results)

In [150]:
results["BPR"]["weight"] 

0.5

## Evaluation

In [37]:
def mean_absolute_error(actual, predicted):
    return np.abs(actual - predicted).mean()

In [38]:
def root_mean_square_error(actual, predicted):
    return np.sqrt(((actual - predicted)**2)).mean()

In [39]:
def precision_at_k(actual, predicted, k):
    return len(
        set(actual) & set(predicted[:k])
    )/k

In [40]:
def recall_at_k(actual, predicted, k):
    return len(
        set(actual) & set(predicted[:k])
    )/len(actual)

In [41]:
def f1_acore_at_k(actual, predicted, k):
    p = precision_at_k(actual, predicted, k)
    r = recall_at_k(actual, predicted, k)
    if p + r == 0:
        return 0
    return 2*(p*r)/(p+r)

In [43]:
def ndcg_at_k():
    pass

### Temp section